参考：https://blog.csdn.net/len9596/article/details/79990823

# 作用域

在python中，作用域分为两种:全局作用域和局部作用域。

**全局作用域**是定义在文件级别的变量, 函数名。而**局部作用域**，则是定义函数内部。

关于作用域，我们要理解：

- a.在全局不能访问到局部定义的变量

- b.在局部能够访问到全局定义的变量，但是不能对全局变量重新赋值；

- 可变类型的全局变量可以在局部代码块被修改

下面我们来看看下面实例:


In [1]:
x = 1
def funx():
    x = 10
    print(x)  # 打印出10

funx()
print(x) # 打印出1

10
1


<br>
如果局部没有定义变量x,那么函数内部会从内往外开始查找x,如果没有找到,就会报错

In [2]:
x = 1
def funx():
    print(x)  

funx()

1


In [3]:
x = 1
def funx():
    def func1():
        print(x)  
    func1()

funx()

1


In [4]:
x = {}

def foo():
    x.update({'a': 1})
    print(x)

foo()
print(x)

{'a': 1}
{'a': 1}


因此,关于作用域的问题，只需要记住两点就行:

全局变量能够被文件任何地方引用，但修改只能在全局进行操作;如果局部没有找到所需的变量，就会往外进行查找，没有找到就会报错.

# 高阶函数（略）

**python 中一切皆对象**

- 函数名可以作为一个值
- 函数名可以作为返回值 
- 函数名可以作为一个参数


# 闭包函数

闭包函数必须满足两个条件:

1. 函数内部定义的函数 
2. 包含对外部作用域而非全局作用域的引用

下面通过一些实例来说明闭包函数:

**实例一:** 以下仅仅在函数内部定义了一个函数,但并非闭包函数.

In [5]:
def outer():
    def inner():
        print("inner func excuted")
    inner()  # 调用执行inner()函数
    print("outer func excuted")
outer()  # 调用执行outer函数


inner func excuted
outer func excuted


**实例二** :以下在函数内部定义了一个函数，而且还引用了一个外部变量x,那么这个是闭包函数么?答案：不是

In [6]:
x = 1

def outer():
    def inner():
        print("x=%s" %x)  # 引用了一个非inner函数内部的变量
        print("inner func excuted")
    inner()  # 执行inner函数
    print("outer func excuted")

outer()

x=1
inner func excuted
outer func excuted


在回头来看看对闭包函数的定义，是不是两条都满足?聪明的你，一定发现不满足第二条.对，这里的变量x，是属于全局变量,而非外部作用于域的变量。再来看看下面例子:

In [7]:
def outer():
    x = 1
    def inner():
        print("x=%s" %x)
        print("inner func excuted")
    inner()
    print("outer func excuted")

outer()

x=1
inner func excuted
outer func excuted


显然,上面实例满足闭包函数的条件。现在，你应该清楚,作为一个闭包函数,必须得满足上述的两个条件,缺一不可。但是,一般情况下，我们都会给闭包函数返回一个值.这里先不说为什么.在接下来的内容中，你会看到这个返回值的用途.

In [8]:
def outer():
    x = 1
    def inner():
        print("x=%s" %x)
        print("inner func excuted")
    print("outer func excuted")
    return inner  # 返回内部函数名
    
outer()

outer func excuted


<function __main__.outer.<locals>.inner()>

现在我们来抽象的定义一下闭包函数。它是函数和与其相关的引用环境组合而成的实体。在实现深约束时，需要创建一个能显式表示引用环境的东西，并将它与相关的子程序捆绑在一起，这样捆绑起成为闭包。在上面实例中，我们可以发现,闭包函数,它必须包含自己的函数以及一个外部变量才能真正称得上是一个闭包函数。如果没有一个外部变量与其绑定,那么這个函数不能算得上是闭包函数。

　　那么怎么知道一个闭包函数有多少个外部引用变量呢?看看下面代码.

In [9]:
def outer():
    x = 1
    y = 2

    def inner():
        print("x= %s" %x)
        print("y= %s" %y)

    print(inner.__closure__)
    return inner

outer()

(<cell at 0x0000022CA3E6EEB8: int object at 0x00007FFF1B889340>, <cell at 0x0000022CA3E6EE58: int object at 0x00007FFF1B889360>)


<function __main__.outer.<locals>.inner()>

结果表明,在inner内部，引用了两个外部局部变量。如果引用的是非局部变量，那么这里输出的为None.

闭包函数的特点：

1. 自带作用域 
2. 延迟计算

　　那么闭包函数有什么作用呢?我们清楚的知道，闭包函数在定义时，一定会绑定一个外部环境。這个整体才能算的上是一个闭包函数，那么我们可以利用这个绑定特性，来完成某些特殊的功能。
  
**实例三**：根据传入的URL,来下载页面源码

In [10]:
import requests

def index(url):
    def get():
        return requests.get(url)
    return get

python = index("https://www.python.org") # 返回的是get函数的地址
print(python()) # 执行get函数《并且将返回的结果打印出来
baidu = index("https://www.baidu.com")
print(baidu())

<Response [200]>
<Response [200]>


有人可以会说,这个不满足闭包函数的条件啊!我没有引用非全局的外部变量啊。其实并非如此,给，我们之前说过,**只要在函数内部的变量都属于函数。那么我在index(url)，这个url也属于函数内部，**只不过我们省略一步而已，所以上面那个函数也是闭包函数。

# 装饰器

装饰器：外部函数传入被装饰函数名，内部函数返回装饰函数名。

特点：
1. 不修改被装饰函数的调用方式 
2. 不修改被装饰函数的源代码

## 简单装饰器

In [11]:
import logging

def use_logging(func):
    def wrapper(*args,**kwargs):
        logging.warning("%s is running"% func.__name__)
        return func(*args,**kwargs)
    wrapper.__doc__ = func.__doc__
    return wrapper


def bar():
    """测试"""
    print('I am bar')

bar = use_logging(bar)
bar()
print(bar.__name__)
print(bar.__doc__)

I am bar
wrapper
测试


### 划重点

函数use_logging就是装饰器，它把执行真正业务方法的func包裹在函数里面，看起来像bar被use_logging装饰了。在这个例子中，函数进入和退出时 ，被称为一个**横切面(Aspect)**，这种编程方式被称为**面向切面的编程(Aspect-Oriented Programming)。**

@符号可以理解为避免再一次赋值操作的 **简写** 

```
@ 等价于 a = d(a)
```

这样我们就可以省去bar = use_logging(bar)这一句了，直接调用bar()即可得到想要的结果。

注意：
1. 原函数的一些元属性会丢失，比如: 文档字符串`__doc__`，需要重新赋值给内部函数。

2. 参数要通过`(*args,**kwargs)` 全部传递过去。

3. 内部函数的返回值，是对被装饰函数简单调用就结束，还是 `return` 其返回值。

In [12]:
@use_logging
def bar(param=2):
    print('param is: {}'.format(param))
    return 'I am bar'


print(bar(param=3))
print(bar.__name__)

param is: 3
I am bar
wrapper


## 有参数装饰器

在上面的装饰器调用中，比如@use_logging，该装饰器唯一的参数就是执行业务的函数。装饰器的语法允许我们在调用时，提供其它参数，比如
```
@decorator(a)
```
这样，就为装饰器的编写和使用提供了更大的灵活性。

在`decorator(a)`阶段 ,就已经发生了一次函数调用，并返回了像上面所说的简单装饰器函数use_logging，所以`decorator` 函数具体实现方法，就是在简单装饰器上再包装一层函数，将简单装饰器作为返回值。

In [13]:
import logging


def use_logging(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                logging.warning("%s is running" % func.__name__)
            return func(*args, **kwargs)

        return wrapper

    return decorator


def foo(name='foo'):
    print('I am bar')


# 不用@ 符号写法
foo = use_logging('warn')(foo)
foo()

I am bar


In [14]:
@use_logging("warn")
def foo(name='foo'):
    print('I am bar')

foo()
print(foo.__name__)

I am bar
wrapper


## 类装饰器

相比函数装饰器，类装饰器具有灵活度大、高内聚、封装性等优点。使用类装饰器还可以依靠类内部的`__call__`方法，当使用 @ 形式将装饰器附加到函数上时，就会调用此方法。

**实例一：**

In [15]:
class Foo(object):
    def __init__(self, func):
        self._func = func
        self.__doc__ = func.__doc__

    def __call__(self):
        print('class decorator runing')
        result = self._func()
        print('class decorator ending')
        return result


@Foo
def bar():
    """test doc"""
    print('bar')


bar()
# print(bar.__doc__)
# print(bar.__class__.__name__)

class decorator runing
bar
class decorator ending


同样的，原函数的元信息不见了，比如函数的docstring、name、参数列表，需要手动传递进去。

实例二：

In [16]:
class Foo(object):
    def __init__(self):
        pass

    def __call__(self, func):
        def _call(*args, **kw):
            print('class decorator runing')
            return func(*args, **kw)

        return _call


class Bar(object):
    @Foo()
    def bar(self, test, ids):   # bar = Foo()(bar)
        print('bar')


Bar().bar('aa', 'ids')

class decorator runing
bar


## 内置装饰器

- @staticmathod
- @classmethod
- @property

# 装饰器的运行顺序: 从下往上
```py
@a
@b
@c
def f ():
```
等效于

```py
f = a(b(c(f)))
```

In [17]:
import time
import random


def timmer(func):
    def wrapper():
        start_time = time.time()
        func()
        stop_time = time.time()
        print('run time is %s' % (stop_time - start_time))

    return wrapper


def auth(func):
    def deco():
        name = input('name: ')
        password = input('password: ')
        if name == 'egon' and password == '123':
            print('login successful')
            func()  #wrapper()
        else:
            print('login err')

    return deco


@auth  # index = auth(timmer(index))
@timmer  # index = timmer(index)
def index():
    time.sleep(3)
    print('welecome to index page')


index()

name: egon
password: 123
login successful
welecome to index page
run time is 3.0000314712524414
